# Angular Component Generator

This notebook leverages an embeddings model to analyze the code structure of this application, and generates a new Angular component with a foundational model that adheres to the proprietary tooling and best practices observed in the existing codebase. 

By using LangChain to manage prompt templates and employing Retrieval-Augmented Generation (RAG) to provide context around embeddings, we ensure that the generated code is both relevant and aligned with the existing codebase. This process demonstrates the capability of fine-tuning a model to follow specific development conventions and incorporate them into the generated code.

To run this notebook, ensure you have access to the following models within Bedrock:
- **Titan Text G1 - Premier**
- **Titan Embeddings G1 - Text**

## Step 1: Environment Setup

This section initializes all libraries and creates an instance of the AWS BedRock boto3 client.

In [ ]:
import json
import os
import sys
import warnings

import boto3

warnings.filterwarnings('ignore')
module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## Step 2: Load and Analyze Angular Components

This loads the Angular components from subdirectories within the `app` folder and uses the embeddings model to analyze their structure.

In [ ]:
from langchain_aws.embeddings import BedrockEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import TextLoader
import os

def load_all_files_recursive(directory):
    documents = []
    subfolders = set()  # Set to store unique subfolder names
    for root, dirs, files in os.walk(directory):
        # Skip hidden subfolders
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        for file in files:
            filepath = os.path.join(root, file)
            try:
                loader = TextLoader(filepath, encoding="utf-8")
                loaded_docs = loader.load()
                # Filter out empty documents
                non_empty_docs = [doc for doc in loaded_docs if doc.page_content.strip()]
                documents.extend(non_empty_docs)
                print(f"Parsed file {filepath}")
            except UnicodeDecodeError:
                print(f"Skipping binary file or file with unknown encoding: {filepath}")
            except Exception as e:
                print(f"Error loading file {filepath}: {e}")

        # Add subfolder names to the set
        for subfolder in dirs:
            subfolders.add(os.path.relpath(os.path.join(root, subfolder), directory))
    return documents, ", ".join(sorted(subfolders))

# Load ALL files recursively
documents, subfolders = load_all_files_recursive("../src/app")
print(f"Loaded {len(documents)} documents.")

# Create embeddings
br_embeddings = BedrockEmbeddings(model_id="amazon.titan-embed-text-v1", client=bedrock_client)

# Create vector store
vectorstore_faiss = FAISS.from_documents(documents, embedding=br_embeddings)
print("Vector store created.")

# List all subfolders
print(f"\nSubfolders: {subfolders}")

# List all documents loaded to the vector store
print("\n--- Documents in Vector Store ---")
for doc in documents:
    print(f"Document Source: {doc.metadata.get('source', 'Unknown')}")
print("--- End of Documents in Vector Store ---")

## Step 3: Generate a New Component

This section uses the foundational model to generate a new Angular component based on the analysis stored in the vector database.

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain_aws import ChatBedrock
import ipywidgets as ipw
from IPython.display import display, clear_output

# Initialize chat model
chat_model = ChatBedrock(model_id="amazon.titan-text-premier-v1:0", client=bedrock_client)

# Define system prompt
system_prompt = (
    "You are an assistant for generating Angular components. "
    "Generate the component code using the following context:"
    "\\n\\n"
    "{context}"
)

# Create prompt template
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)

# Create retrieval chain
retriever = vectorstore_faiss.as_retriever()
question_answer_chain = create_stuff_documents_chain(chat_model, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# Define ChatUX class
class ChatUX:
    """ A chat UX using IPWidgets """
    def __init__(self, rag_chain):
        self.rag_chain = rag_chain
        self.name = None
        self.btn = None
        self.out = ipw.Output()

    def start_chat(self):
        print("Welcome to the component generator!")
        display(self.out)
        self.chat(None)

    def inject_prompt(self, name):
        return (
            "Create a component named " + name + ". "
            "Each component must have 5 files: "
            "<name>.component.ts - containing the primary component code, "
            "<name>.config.ts - containing the config data, "
            "<name>.component.html - containing the HTML markup, "
            "<name>.component.css - stylesheets, "
            "<name>.component.spec.ts - unit tests. "
            "Analyze the files within the subfolders: " + subfolders + " "
            "And use them as examples on how each file is referenced from another. "
            "Strictly follow the following response template within ```"
            "```"
            "//<filename>"
            "<generated code>"
            "\\n\\n"
            "```"
            "Do not respond back with any additional information."
        )

    def chat(self, _):
        if self.name is None:
            prompt = ""
        else:
            prompt = self.inject_prompt(self.name.value)
            
        if 'q' == prompt or 'quit' == prompt or 'Q' == prompt:
            with self.out:
                print("Thank you, that was a nice chat !!")
            return
        elif len(prompt) > 0:
            with self.out:
                thinking = ipw.Label(value="Thinking...")
                display(thinking)

                try:
                    response = self.rag_chain.invoke({"input": prompt})
                    result = response['answer']
                except:
                    result = "Something went wrong!"
                    
                thinking.value = ""
                print(f"AI: {result}")
                self.name.disabled = True
                self.btn.disabled = True
                self.name = None

        if self.name is None:
            with self.out:
                self.name = ipw.Text(description="You:", placeholder='q to quit')
                self.btn = ipw.Button(description="Send")
                self.btn.on_click(self.chat)
                display(ipw.Box(children=(self.name, self.btn)))

# Start chat to get component name
chat = ChatUX(rag_chain)
chat.start_chat()

## Conclusion

Through adequate tuning, the model successfully generated code that adheres to the established boilerplate. This approach can be seamlessly extended to various types of codebases, including web, mobile, APIs, and more.